<a href="https://colab.research.google.com/github/miguelcasadinho/colab/blob/main/soccer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data cleaning

In [1]:
# Import the libraries
import numpy as np
import pandas as pd

In [6]:

# Import the dataset
dataset = pd.read_csv('games.csv')

# Filter by league
dataset = dataset[dataset['competition_id'].isin(['ES1'])]

# Keep only specific columns
columns_to_keep = ['season','round','date','home_club_name','away_club_name','home_club_position','away_club_position','home_club_goals','away_club_goals']
dataset = dataset[columns_to_keep]

# Create a new column 'winner'
dataset['winner'] = dataset.apply(
    lambda row: row['home_club_name'] if row['home_club_goals'] > row['away_club_goals']
    else row['away_club_name'] if row['home_club_goals'] < row['away_club_goals']
    else 'draw', axis=1
)


print(dataset.head)

<bound method NDFrame.head of        season         round        date                       home_club_name  \
75       2013   4. Matchday  2013-09-16                 Athletic Club Bilbao   
76       2013  25. Matchday  2014-02-23     Valencia Club de Fútbol S. A. D.   
77       2013   9. Matchday  2013-10-21     Real Club Celta de Vigo S. A. D.   
78       2013  18. Matchday  2014-01-04     Valencia Club de Fútbol S. A. D.   
79       2013  14. Matchday  2013-11-23       Club Atlético de Madrid S.A.D.   
...       ...           ...         ...                                  ...   
71810    2018   1. Matchday  2018-08-19      Rayo Vallecano de Madrid S.A.D.   
71856    2023  18. Matchday  2023-12-21                             Cádiz CF   
71857    2023  12. Matchday  2023-11-04     Real Club Celta de Vigo S. A. D.   
71864    2022  25. Matchday  2023-03-12  Real Club Deportivo Mallorca S.A.D.   
71866    2023  33. Matchday  2024-04-28     Villarreal Club de Fútbol S.A.D.   

         